## A Continual Learning Model

A Continual Learning Model built with Avalanche.

In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive, EWC
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

In [2]:
model=SimpleMLP(num_classes=10)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
model

SimpleMLP(
  (features): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (classifier): Linear(in_features=512, out_features=10, bias=True)
)

In [5]:
perm_mnist=PermutedMNIST(n_experiences=3,dataset_root='E:/Datasets/')

In [6]:
train_stream=perm_mnist.train_stream
test_stream=perm_mnist.test_stream

In [7]:
train_stream

In [8]:
optimizer=Adam(model.parameters())
criterion=CrossEntropyLoss()
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger])

c:\users\ashwin\miniconda3\lib\site-packages\avalanche\training\plugins\evaluation.py:75: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


In [9]:
strategy=EWC(model,optimizer,criterion,train_mb_size=32,train_epochs=2,eval_mb_size=32,ewc_lambda=0.001,device=device,evaluator=eval_plugin)

In [10]:
results=[]
for train_task in train_stream:
    print(train_task.classes_in_this_experience)
    print(train_task.current_experience)
    strategy.train(train_task)
    results.append(strategy.eval(test_stream))
    break

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:28<00:00, 66.72it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2721
	Loss_MB/train_phase/train_stream/Task000 = 0.1614
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9169
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9375
100%|██████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:26<00:00, 69.94it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1666
	Loss_MB/train_phase/train_stream/Task000 = 0.0806
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9502
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9688
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|█████████████████████████

In [11]:
model

SimpleMLP(
  (features): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (classifier): Linear(in_features=512, out_features=10, bias=True)
)

In [12]:
train_stream

In [13]:
perm_mnist

In [14]:
results

[{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.96875,
  'Loss_MB/train_phase/train_stream/Task000': 0.08061923831701279,
  'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.95015,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.16664849396621187,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9699,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.0995846578541561,
  'Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001': 0.117,
  'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 4.93001266708374,
  'Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002': 0.1371,
  'Loss_Exp/eval_phase/test_stream/Task002/Exp002': 7.461552218627929,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.9699,
  'Top1_Acc_Stream/eval_phase/test_stream/Task001': 0.117,
  'Top1_Acc_Stream/eval_phase/test_stream/Task002': 0.1371,
  'Loss_Stream/eval_phase/test_stream/Task000': 0.0995846578541561,
  'Loss_Stream/eval_phase/test_stream/Task001': 4.93001266708374,
  'Loss_Stream/eval_phase